<a href="https://colab.research.google.com/github/anderfrago/trasladar-notas/blob/main/2021_CompartirCompetenciasClave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@markdown # TRASLADAR NOTAS 

#@markdown ---------

#@markdown ## INFORMACIÓN REQUERIDA
#@markdown ### Introduce nombre del archivo: 
nombre_excel_notas = "Copia CC 2Dam" #@param {type:"string"} 
#@markdown ### Introduce la ruta al directorio: 
#@markdown Introduce la **ubicación donde se encuentra el archivo** de notas para trasladar a los alumnos.
ruta_directorio = "4Vientos/2021/Notak/pro b@k" #@param {type:"string"} 
path =  "/content/drive/MyDrive/"+ruta_directorio
filenombre_excel_notas = nombre_excel_notas+'.xlsx'

#@markdown ## Columna nombre y correo: 
#@markdown ### Introduce la columna con los nombres de los alumnos
letra_columna_nombre =  "B"#@param {type:"string"} 
#@markdown ### Introduce la columna con los correos de los alumnos
letra_columna_correo =  "C"#@param {type:"string"} 

#@markdown ## Cabecera: 
#@markdown ### Introduce el número de líneas de cabecera: 
numero_cabecera =  7#@param {type:"number"} 

#@markdown --------------------

#@markdown ## OPCIONES
#@markdown ### Introduce nombre de la hoja/pestaña del documento: 
#@markdown *En caso de no especificar se traslada la primera pestaña*
nombre_hoja = "NOTAS" #@param {type:"string"} 
#@markdown ### Selecciona el tipo de archivo.
#@markdown Excel para XLSX y Google Sheet para GSheet
extension = "Google Sheet" #@param [ "Excel", "Google Sheet"] {type:"string"}
#@markdown ### Introduce el nombre del archivo plantilla: 
#@markdown En caso de hacer uso de columnas colapsables es requisito crear una plantilla.
#@markdown *Consiste en crear una copia del documento sin filas de alumnos.*
plantilla_cabecera =  ""#@param {type:"string"} 



# Get positional value from Sheet column
def positionfromcolumn(l) :
  num_array = [ord(x.upper()) - 64 for x in l]
  result = 0
  for i, v in enumerate(num_array):
    result = i*26+v
  return result

col_nombre = positionfromcolumn(letra_columna_nombre)
#print(col_nombre)
col_correo = positionfromcolumn(letra_columna_correo)
#print(col_correo)

import requests
import os
from google.colab import drive as colabdrive
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Copy grades to excel
!pip install openpyxl > /dev/null
import openpyxl
import pandas as pd
import time
import shutil
from openpyxl import load_workbook
from copy import copy
import numpy as np


class TransferGrades:

  def __init__(self):
      # Google Authentication
      self._gauth = None
      # Google Drive Python
      self._drive = None
      # Colab Drive
      self._colabdrive = None

  @property
  def drive(self):
      return self._drive

  @drive.setter
  def drive(self, value):
      self._drive = value

  @drive.deleter
  def drive(self):
      del self._drive

  @property
  def gauth(self):
      return self._gauth

  @gauth.setter
  def gauth(self, value):
      self._gauth = value

  @gauth.deleter
  def gauth(self):
      del self._gauth

  @property
  def colabdrive(self):
      return self._colabdrive

  @colabdrive.setter
  def colabdrive(self, value):
      self._colabdrive = value

  @colabdrive.deleter
  def colabdrive(self):
      del self._colabdrive

  def connectgdrive(self): 
    # colabdrive.flush_and_unmount()
    colabdrive.mount('/content/drive', force_remount=True)

  def sdkaccess(self):  
    # Launch Google Authentication for first time
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    return drive, gauth

  def search(self, drive, gauth):    
    if extension == "Excel" :
      print("Búsqueda de un documento de tipo Microsoft Excel: ",filenombre_excel_notas)
      listed = drive.ListFile({'q': " title contains '"+filenombre_excel_notas+"'"}).GetList()
    else :
      print("Búsqueda de un documento de tipo GSheet: ", nombre_excel_notas)
      listed = drive.ListFile({'q': " mimeType ='application/vnd.google-apps.spreadsheet' and title contains '"+nombre_excel_notas+"'"}).GetList()
    try:

      if len(listed) :
        # Last modified file
        file = listed[0]
        #print(file['id'])
        spreadsheetId = file['id']
        print("Archivo de notas localizado exitosamente!")
        if extension == "Excel" :
          downloaded = drive.CreateFile({'id': spreadsheetId})
          downloaded.GetContentFile(filenombre_excel_notas)
        else :
          url = 'https://docs.google.com/spreadsheets/d/' + spreadsheetId + '/export?format=xlsx&id=' + spreadsheetId
          headers = {'Authorization': 'Bearer ' + gauth.credentials.access_token}
          res = requests.get(url, headers=headers)
          with open(filenombre_excel_notas, 'wb') as f:
              f.write(res.content)
      else :
        print("No se ha localizado el archivo ",filenombre_excel_notas) 
        print("\t - Comprueba que el nombre del archivo es correcto en INFORMACIÓN REQUERIDA, nombre_excel_notas")  
        print("\t - Comprueba que tipo del archivo es correcto en OPCIONES: Excel para XLS o Google Sheet para GSheet")  
        print("\t En caso de corregir el valor debes volver a ejecutar la primera celda.")   
    except Exception as e:
      print("ERROR!: Revisa el archivo de logs. Exception: "+ str(e))
      with open('trasladar-notas.log', 'w') as f:
        f.write("ERROR: search ::"+ filenombre_excel_notas +" Exception: "+ str(e))

  def copygrades(self):
    ## openpyxl BUG :: Calculateing formula values
    # When using formulas in Sheets openpyxl does not evaluate the results
    # that is why panda dataframe are used to transfer 
    # values to the result file
    ##
    os.chdir('/content')
    # Work Book used to copy formats and comments
    wb = openpyxl.load_workbook(filenombre_excel_notas)
    # Select Worksheet by name 
    if nombre_hoja :
      print("\t Selección de pestaña por nombre: ", nombre_hoja)
      ws_original = wb[nombre_hoja]
      # Grade values are read from dataframe instead of Excel to resolve excel formulas
      df = pd.read_excel(open(filenombre_excel_notas,'rb'),header=None,sheet_name=nombre_hoja)
    else :
      # Select First Worksheet
      print("\t seleccionada la primera pestaña del documento")
      ws_original = wb.worksheets[0]
      # Grade values are read from dataframe instead of Excel to resolve excel formulas
      df = pd.read_excel(open(filenombre_excel_notas,'rb'),header=None)

    ## Process of transfering notes to students folder
    ###
    print("Generando las carpetas...")
    for index, row in df.iloc[numero_cabecera:].iterrows():

      # Get name to create the folder and contact mail
      name = row.values[col_nombre-1]  
      email = row.values[col_correo-1]
      # Creation of students folder
      print("\t - Trasladando notas al alumn@  "+str(name))
      try: 
        os.chdir(path)
        if not os.path.isdir(name) :
            print("\t\t Primera vez que se trasladan las notas, creación del directorio del alumn@")
            os.mkdir(name)
            time.sleep(1)

        os.chdir(name)
        # Transform from numpy.ndarray to excel values
        row_list = row.values.tolist()
        file_name = name+"__"+filenombre_excel_notas
        if plantilla_cabecera :
          print("\t\t Copiando la plantilla de cabecera")
          ## COPY TEMPLATE
          template=path+"/"+plantilla_cabecera+".xlsx"
          shutil.copyfile(template, file_name)          
          time.sleep(1)
          # Load excel
          wb = openpyxl.load_workbook(file_name)
          ws = wb.worksheets[0]
        else :
          # Creation of excel
          wb = openpyxl.Workbook()
          wb.save(file_name)
          ws = wb.worksheets[0]
          ## COPY HEADER
          print("\t\t Generando cabecera")
          for index_header, row_header in df.iloc[:numero_cabecera].iterrows():
            row_list_header = row_header.values.tolist()
            for idx_header, val_header in enumerate(row_list_header):
              ws.cell(row=index_header+1, column=idx_header+1).value =val_header
              ws.cell(row=index_header+1, column=idx_header+1).comment = ws_original.cell(row=index_header+1, column=idx_header+1).comment
              if ws_original.cell(row=index_header+1, column=idx_header+1).has_style:
                    ws.cell(row=index_header+1, column=idx_header+1).font = copy(ws_original.cell(row=index_header+1, column=idx_header+1).font)
                    ws.cell(row=index_header+1, column=idx_header+1).border = copy(ws_original.cell(row=index_header+1, column=idx_header+1).border)
                    ws.cell(row=index_header+1, column=idx_header+1).fill = copy(ws_original.cell(row=index_header+1, column=idx_header+1).fill)
                    ws.cell(row=index_header+1, column=idx_header+1).number_format = copy(ws_original.cell(row=index_header+1, column=idx_header+1).number_format)
                    ws.cell(row=index_header+1, column=idx_header+1).protection =copy( ws_original.cell(row=index_header+1, column=idx_header+1).protection)
                    ws.cell(row=index_header+1, column=idx_header+1).alignment = copy(ws_original.cell(row=index_header+1, column=idx_header+1).alignment)

        ## ADDING STUDENTS VALUES 
        #print(row_list)
        print("\t\t Copiando notas")
        for idx, val in enumerate(row_list):
          ws.cell(row=numero_cabecera+1, column=idx+1).value = val
          ws.cell(row=numero_cabecera+1, column=idx+1).comment =ws_original.cell(row=index+1, column=idx+1).comment
          if ws_original.cell(row=numero_cabecera+1, column=idx+1).has_style:
            ws.cell(row=numero_cabecera+1, column=idx+1).font = copy(ws_original.cell(row=index+1, column=idx+1).font)
            ws.cell(row=numero_cabecera+1, column=idx+1).border = copy(ws_original.cell(row=index+1, column=idx+1).border)
            ws.cell(row=numero_cabecera+1, column=idx+1).fill = copy(ws_original.cell(row=index+1, column=idx+1).fill)
            ws.cell(row=numero_cabecera+1, column=idx+1).number_format = copy(ws_original.cell(row=index+1, column=idx+1).number_format)
            ws.cell(row=numero_cabecera+1, column=idx+1).protection = copy(ws_original.cell(row=index+1, column=idx+1).protection)
            ws.cell(row=numero_cabecera+1, column=idx+1).alignment = copy(ws_original.cell(row=index+1, column=idx+1).alignment)
      
        wb.save(file_name)
        wb.close()
      except Exception as e:
        os.chdir('/content')
        print("ERROR!: Revisa el archivo de logs. Exception: "+ str(e))
        with open('trasladar-notas.log', 'w') as f:
          f.write("ERROR: copygrades :: Exception: "+ str(e))
    #return df

  def share(self, drive, gauth):
    ###
    ## Process of transfering notes to students folder
    ###
    os.chdir('/content')
    if nombre_hoja:
      df = pd.read_excel(open(filenombre_excel_notas,'rb'),header=None,sheet_name=nombre_hoja)
    else :
      df = pd.read_excel(open(filenombre_excel_notas,'rb'),header=None)
    
    for index, row in df.iloc[numero_cabecera:].iterrows():
      # Get name to create the folder and contact mail
      name = row.values[col_nombre-1]  
      email = row.values[col_correo-1].strip()
      print(name)
      # Creation of students folder
      os.chdir(path)
      #print(os.getcwd())
      #os.chdir(name)
      # Transform from numpy.ndarray to excel values
      row_list = row.values.tolist()
      # file_name = name+filenombre_excel_notas
      listed = drive.ListFile({'q': "title: '"+name+"'"}).GetList()
      # Get las modified version
      file = listed[0]
      print("Compartiendo notas con "+email)

      try:
        file.InsertPermission({'type': 'user','value': email,'role': 'writer'})
      except Exception as e:
        print("ERROR!: Revisa el archivo de logs. Exception: "+ str(e))
        with open('trasladar-notas.log', 'w') as f:
          f.write("ERROR: share :: An exception occurred with: "+email+" Exception: "+ str(e))



##
# Application starting point
## 
if 'google.colab' in str(get_ipython()):
  app = TransferGrades()
  print("Funcionalidades inicializadas...")
else:
  print('CoLab no está arrancado')


# VALIDAR CUENTA GOOGLE

In [ ]:

#@markdown Para usar la aplicación es necesario validar la cuenta de Google 2 veces.
#@markdown * La primera para instalar GDrive en la máquina en la nube.
#@markdown * La segunda para usar las herramientas de desarrollo de GCloud.

try :
  if app.colabdrive is None :
    print("NO SE NECESITA VALIDACIÓN")
    print("ya está instalado Google Drive en el PC en la nube de Colab.")
  else :
    print("COMIENZA EL PROCESO DE VALIDACIÓN DE LA CUENTA GOOGLE PARA INSTALAR GOOGLE DRIVE")
    print("A continuación debes acceder a la URL que aparezca")
    print("Despues, valida tu cuenta y pega el código de confirmación en la cajetilla")
    app.connectgdrive()
    print("FINALIZA EL PROCESO DE VALIDACIÓN DE LA CUENTA GOOGLE")
except Exception as e:
  print("ERROR: Conectando a Google Drive. Exception: "+ str(e))
  with open('trasladar-notas.log', 'w') as f:
    f.write("ERROR: connectgdrive :: Exception: "+ str(e))

try:
  if app.drive is None and app.gauth is None :
    print("NO SE NECESITA VALIDACIÓN")
    print("ya está instalado Google Cloud SDK en el PC en la nube de Colab.")
  else :
    print("COMIENZA EL PROCESO DE VALIDACIÓN DE LA CUENTA GOOGLE PARA INSTALAR EL SDK DE GOOGLE CLOUD")
    print("Repite el proceso de validación para dar acceso al SDK de Google Drive")
    drive, gauth = app.sdkaccess()
    print("FINALIZA EL PROCESO DE VALIDACIÓN DE LA CUENTA GOOGLE PARA INSTALAR EL SDK DE GOOGLE CLOUD")
except Exception as e:
  print("ERROR: Obteniendo permisos de acceso al SDK de Google Drive. Exception: "+ str(e))
  with open('trasladar-notas.log', 'w') as f:
    f.write("ERROR: sdkaccess :: Exception: "+ str(e))



# GENERAR NOTAS PARA COMPARTIR

In [ ]:
#@markdown ## Buscar fichero de notas en Google Drive
print("COMIENZA EL PROCESO DE BUSCAR EL ARCHIVO DE NOTAS EN GOOGLE DRIVE")
app.search(drive, gauth)
print("FINALIZA EL PROCESO DE BUSCAR EL ARCHIVO DE NOTAS EN GOOGLE DRIVE")

In [ ]:
#@markdown ## Copiar las notas al alumando
#@markdown En la carpeta se genera un excel y se copian las líneas de cabecera junto la nota del alumno
print("COMIENZA EL PROCESO DE GENERAR LAS CARPETA DE GOOGLE DRIVE CON EL EXCEL PARA EL ALUMNADO")
app.copygrades()
print("FINALIZA EL PROCESO DE GENERAR LAS CARPETA DE GOOGLE DRIVE CON EL EXCEL PARA EL ALUMNADO")

# ENVIAR NOTIFICACIÓN


In [ ]:
#@markdown ## Compartir las carpetas con los alumnos
#@markdown Se comparte la carpeta con nombre del alumn@ y se notificará al alumnado vía mail
print("COMIENZA EL PROCESO DE COMPARTIR LAS CARPETAS GOOGLE DRIVE CON EL ALUMNADO")
app.share(drive, gauth)
print("FINALIZA EL PROCESO DE COMPARTIR LAS CARPETAS GOOGLE DRIVE CON EL ALUMNADO")

# REINICIAR MÁQUINA



In [ ]:
#@markdown En caso de querer trasladar multiples documentos

#Kill previus machine
quit()